# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [61]:
import keras
import numpy as np
import pandas as pd
from keras import optimizers
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

np.random.seed(7)  # fixed for reproducibility

In [2]:
# Load the data
cust_churn_df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [3]:
# Inspect data
pd.set_option("display.max_columns", 30)
# cust_churn_df.isna().sum().sum(), cust_churn_df.head()

In [4]:
# Drop 2 columns; label encode; then define input and target
cust_churn_df = cust_churn_df.drop('customerID', axis=1)
label_enc = LabelEncoder()
cust_churn_df = cust_churn_df.apply(label_enc.fit_transform)
X = cust_churn_df.drop('Churn', axis=1)
y = cust_churn_df['Churn']

In [5]:
# Scale input data, if needed
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [36]:
# For hyperparameter tuning, eg, optimizers
sgd = optimizers.SGD(lr=0.01, momentum=0.0, nesterov=False) 
# default lr is 0.01; 0.1 did not improve; 0.2 worsened
# default momentum is 0.0; 0.1 did not improve; 0.2 did not improve
# momentum=0.1, nesterov=True resulted in no change

adam = optimizers.Adam(lr=0.001, amsgrad=False)  # default lr=0.001; 0.01 and 0.1 caused no change
# amsgrad variant of adam caused no change

In [70]:
# Train baseline model with 24 input nodes, 1 hidden layer of sixteen nodes, and 1 output node
# See adjacent comments for manual tuning

# Global hyperparameters
inputs = X.shape[1]
epochs = 20  # increasing to 50 did not improve acc
batch_size = 80  # baseline is 30; increase to 64 did not improve acc
# Create model
model = Sequential()
model.add(Dense(24, activation='relu', input_shape=(inputs,)))  # increasing nodes to 30 or 64 did not improve acc
# model.add(Dropout(0.2))  # addition of this layer at 0.1 rate caused no change; same for 0.2 rate
model.add(Dense(16, activation='relu'))  # 16 nodes: val_loss: 0.2704 - val_acc: 0.7296; not improved w 24 nodes
# model.add(Dense(11, activation='relu'))  adding this layer did not improve accuracy, so try increasing prev nodes
model.add(Dense(1, activation='sigmoid'))  # softmax dramatically worsened acc; relu caused no change
# Compile
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
# binary_crossentropy loss caused no change; sgd optimizer, tuned, caused no improvement
# Fit model
model.summary()
model.fit(X, y, validation_split=0.2, epochs=epochs, batch_size=batch_size)  # val_split at 0.1 caused slight decr
# validation_split at 0.33 also caused slight loss of acc
# scores = model.evaluate()
# print(f'{model.metrics[0]}: {scores[1]*100}')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_218 (Dense)            (None, 24)                480       
_________________________________________________________________
dense_219 (Dense)            (None, 16)                400       
_________________________________________________________________
dense_220 (Dense)            (None, 1)                 17        
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________
Train on 5634 samples, validate on 1409 samples
Epoch 1/20
5634/5634 [==============================] - 6s 1ms/step - loss: 0.2979 - acc: 0.7015 - val_loss: 0.2863 - val_acc: 0.7126
Epoch 2/20
5634/5634 [==============================] - 0s 78us/step - loss: 0.2797 - acc: 0.7199 - val_loss: 0.2870 - val_acc: 0.7126
Epoch 3/20
5634/5634 [==============================] - 0s 80us/step - loss: 0.2689 - acc: 

In [74]:
# Bring in GridSearchCV - credit RA/LSDS

# Create models
# Function to create model, required for KerasClassifier


def create_model():
    # Create model
    model = Sequential()
    model.add(Dense(24, input_dim=19, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


kc_model = KerasClassifier(build_fn=create_model, verbose=1)  # default solver=adam

# Evaluate using 10-fold cross validation
# kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
# results = cross_val_score(kc_model, X, y, cv=kfold)
# print('Mean of accuracy results after 10-fold cross validation:', results.mean(), '\n')

# Define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# Define the grid search parameters
param_grid = {'batch_size': [20, 40, 60, 80],
              'epochs': [20]}
params_activation_batch = {'activation': ['relu', 'sigmoid', 'tanh'],  # activation search not an option?
                           'batch_size': [20, 40, 60],
                           'epochs': [20]}
# Create Grid Search
grid = GridSearchCV(estimator=kc_model, param_grid=param_grid, n_jobs=1, cv=5)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/20
5634/5634 [==============================] - 9s 2ms/step - loss: 4.2483 - acc: 0.7346
Epoch 2/20
5634/5634 [==============================] - 2s 386us/step - loss: 4.2780 - acc: 0.7345
Epoch 3/20
5634/5634 [==============================] - 2s 386us/step - loss: 4.2772 - acc: 0.7345
Epoch 4/20
5634/5634 [==============================] - 2s 380us/step - loss: 4.2703 - acc: 0.7334
Epoch 5/20
5634/5634 [==============================] - 2s 400us/step - loss: 4.2949 - acc: 0.7299
Epoch 6/20
5634/5634 [==============================] - 3s 450us/step - loss: 4.2596 - acc: 0.7332
Epoch 7/20
5634/5634 [==============================] - 2s 398us/step - loss: 4.2783 - acc: 0.7345
Epoch 8/20
5634/5634 [==============================] - 2s 398us/step - loss: 4.2775 - acc: 0.7345
Epoch 9/20
5634/5634 [==============================] - 2s 404us/step - loss: 4.2768 - acc: 0.7345
Epoch 10/20
5634/5634 [==============================] - 2s 392us/step - loss: 4.2551 - acc: 0.7330
Epoch 11/20

5635/5635 [==============================] - 2s 442us/step - loss: 4.1515 - acc: 0.7381
Epoch 20/20
5635/5635 [==============================] - 2s 328us/step
Epoch 1/20
5635/5635 [==============================] - 9s 2ms/step - loss: 4.2196 - acc: 0.7343
Epoch 2/20
5635/5635 [==============================] - 2s 417us/step - loss: 4.2565 - acc: 0.7358
Epoch 3/20
5635/5635 [==============================] - 2s 415us/step - loss: 4.2549 - acc: 0.7358
Epoch 4/20
5635/5635 [==============================] - 2s 413us/step - loss: 4.2540 - acc: 0.7350
Epoch 5/20
5635/5635 [==============================] - 2s 414us/step - loss: 10.4898 - acc: 0.3382
Epoch 6/20
5635/5635 [==============================] - 2s 417us/step - loss: 6.1829 - acc: 0.6131
Epoch 7/20
5635/5635 [==============================] - 2s 433us/step - loss: 4.2591 - acc: 0.7358
Epoch 8/20
5635/5635 [==============================] - 2s 412us/step - loss: 4.2591 - acc: 0.7358
Epoch 9/20
5635/5635 [============================

5634/5634 [==============================] - 1s 209us/step - loss: 0.5601 - acc: 0.7646
Epoch 17/20
5634/5634 [==============================] - 1s 244us/step - loss: 0.6778 - acc: 0.7467
Epoch 18/20
5634/5634 [==============================] - 2s 315us/step - loss: 0.6291 - acc: 0.7529
Epoch 19/20
5634/5634 [==============================] - 2s 290us/step - loss: 0.6158 - acc: 0.7542
Epoch 20/20
5634/5634 [==============================] - 1s 190us/step
Epoch 1/20
5635/5635 [==============================] - 9s 2ms/step - loss: 4.2535 - acc: 0.7356
Epoch 2/20
5635/5635 [==============================] - 1s 208us/step - loss: 4.2150 - acc: 0.7347
Epoch 3/20
5635/5635 [==============================] - 1s 228us/step - loss: 4.1803 - acc: 0.7375
Epoch 4/20
5635/5635 [==============================] - 1s 230us/step - loss: 4.1527 - acc: 0.7363
Epoch 5/20
5635/5635 [==============================] - 1s 225us/step - loss: 4.1483 - acc: 0.7379
Epoch 6/20
5635/5635 [==========================

5634/5634 [==============================] - 1s 112us/step - loss: 3.6671 - acc: 0.7387
Epoch 14/20
5634/5634 [==============================] - 1s 105us/step - loss: 3.6594 - acc: 0.7355
Epoch 15/20
5634/5634 [==============================] - 1s 109us/step - loss: 3.7281 - acc: 0.7133
Epoch 16/20
5634/5634 [==============================] - 1s 104us/step - loss: 4.1209 - acc: 0.7338
Epoch 17/20
5634/5634 [==============================] - 1s 103us/step - loss: 3.9726 - acc: 0.7288
Epoch 18/20
5634/5634 [==============================] - 1s 128us/step - loss: 4.3034 - acc: 0.7329
Epoch 19/20
5634/5634 [==============================] - 1s 134us/step - loss: 4.3031 - acc: 0.7329
Epoch 20/20
5634/5634 [==============================] - 1s 99us/step
Epoch 1/20
5634/5634 [==============================] - 9s 2ms/step - loss: 4.2871 - acc: 0.7339
Epoch 2/20
5634/5634 [==============================] - 1s 133us/step - loss: 4.2868 - acc: 0.7339
Epoch 3/20
5634/5634 [========================

5634/5634 [==============================] - 1s 109us/step - loss: 3.7591 - acc: 0.7364
Epoch 11/20
5634/5634 [==============================] - 1s 102us/step - loss: 4.2017 - acc: 0.7357
Epoch 12/20
5634/5634 [==============================] - 1s 111us/step - loss: 4.0942 - acc: 0.7368
Epoch 13/20
5634/5634 [==============================] - 1s 115us/step - loss: 3.8592 - acc: 0.7330
Epoch 14/20
5634/5634 [==============================] - 1s 131us/step - loss: 3.8466 - acc: 0.7293
Epoch 15/20
5634/5634 [==============================] - 1s 131us/step - loss: 3.8705 - acc: 0.7322
Epoch 16/20
5634/5634 [==============================] - 1s 137us/step - loss: 3.7236 - acc: 0.7345
Epoch 17/20
5634/5634 [==============================] - 1s 181us/step - loss: 3.7062 - acc: 0.7311
Epoch 18/20
5634/5634 [==============================] - 1s 162us/step - loss: 4.2749 - acc: 0.7345
Epoch 19/20
5634/5634 [==============================] - 1s 160us/step - loss: 4.2747 - acc: 0.7345
Epoch 20/20


5635/5635 [==============================] - 1s 118us/step - loss: 0.8331 - acc: 0.7391
Epoch 8/20
5635/5635 [==============================] - 1s 133us/step - loss: 0.6265 - acc: 0.7505
Epoch 9/20
5635/5635 [==============================] - 1s 151us/step - loss: 0.6287 - acc: 0.7496
Epoch 10/20
5635/5635 [==============================] - 1s 119us/step - loss: 0.5876 - acc: 0.7569
Epoch 11/20
5635/5635 [==============================] - 1s 156us/step - loss: 0.5865 - acc: 0.7542
Epoch 12/20
5635/5635 [==============================] - 1s 139us/step - loss: 0.6247 - acc: 0.7494
Epoch 13/20
5635/5635 [==============================] - 1s 147us/step - loss: 0.6066 - acc: 0.7468
Epoch 14/20
5635/5635 [==============================] - 1s 143us/step - loss: 0.5446 - acc: 0.7650 0s - loss: 0.6093 
Epoch 15/20
5635/5635 [==============================] - 1s 147us/step - loss: 0.5469 - acc: 0.7512
Epoch 16/20
5635/5635 [==============================] - 1s 146us/step - loss: 0.5022 - acc: 0.

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimization tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?